In [2]:
import pandas as pd
from modules import data
from modules import metrics
from modules import valuation

In [20]:
# define parameters
CURRENT_PRICE = 86.00
BETA = 0.74

CAP_GROWTH = 1.15
MARKET_RETURN = 0.1
RISK_FREE_RETURN = 0.035

In [21]:
#get data
df = data.get_data()

In [22]:
# df = df.iloc[1:]

In [23]:
#get metrics
SHARE_COUNT = metrics.shares(df)
WACC = metrics.wacc(df,
                     CURRENT_PRICE,
                     market_return_rate=MARKET_RETURN,
                     risk_free_rate=RISK_FREE_RETURN, 
                     beta=BETA)


# get past 4 years performance history

In [24]:
OCF_History = list(df.iloc[:4]["Operating_Cash_Flow"])
Net_Income_History = list(df.iloc[:4]["Net_Income_Common"])
FCF_History = list(df.iloc[:4]["Free_Cash_Flow"])
Dividends_History = list(df.iloc[:4]["dividend_per_share"])

In [25]:
# Net_Income_History[-1] = 0.01

# Valuations

### Operating cash flow

In [26]:
OCF_YOY_GROWTH = valuation.get_ave_yoy_growth_DCF_value(history=OCF_History,
                                       cap_growth=CAP_GROWTH, 
                                       wacc=WACC, 
                                       no_of_shares=SHARE_COUNT)

OCF_CAGR_GROWTH = valuation.get_cagr_growth_DCF_value(history=OCF_History,
                                    cap_growth=CAP_GROWTH, 
                                    wacc=WACC, 
                                    no_of_shares=SHARE_COUNT)

In [27]:
print(f"""Value based on YOY OCF projection: {OCF_YOY_GROWTH}
Value based on CAGR OCF projection: {OCF_CAGR_GROWTH}""")

Value based on YOY OCF projection: 124.21608365092844
Value based on CAGR OCF projection: 110.55515749880917


### Net income

In [28]:
NI_YOY_GROWTH = valuation.get_ave_yoy_growth_DCF_value(history=Net_Income_History,
                                       cap_growth=CAP_GROWTH, 
                                       wacc=WACC, 
                                       no_of_shares=SHARE_COUNT)

NI_CAGR_GROWTH = valuation.get_cagr_growth_DCF_value(history=Net_Income_History,
                                    cap_growth=CAP_GROWTH, 
                                    wacc=WACC, 
                                    no_of_shares=SHARE_COUNT)

problem occured. Base year is negative value


In [29]:
print(f"""Value based on YOY Net Income projection: {NI_YOY_GROWTH}
Value based on CAGR Net Income projection: {NI_CAGR_GROWTH}""")

Value based on YOY Net Income projection: 41.45855418554079
Value based on CAGR Net Income projection: 0.0


### FCF 

In [30]:
FCF_YOY_GROWTH = valuation.get_ave_yoy_growth_DCF_value(history=FCF_History,
                                       cap_growth=CAP_GROWTH, 
                                       wacc=WACC, 
                                       no_of_shares=SHARE_COUNT)

FCF_CAGR_GROWTH = valuation.get_cagr_growth_DCF_value(history=FCF_History,
                                    cap_growth=CAP_GROWTH, 
                                    wacc=WACC, 
                                    no_of_shares=SHARE_COUNT)
FCF_PERPETUAL = valuation.perpetual_FCF_growth_valuation(history = FCF_History, 
                                                        wacc=WACC, 
                                                        no_of_shares=SHARE_COUNT)

In [31]:
print(f"""Value based on YOY FCF projection: {FCF_YOY_GROWTH}
Value based on CAGR FCF projection: {FCF_CAGR_GROWTH}
Value based on perpetual growth projectionL {FCF_PERPETUAL}""")

Value based on YOY FCF projection: 102.9411542183842
Value based on CAGR FCF projection: 87.56519706655166
Value based on perpetual growth projectionL 33.72899249539686


In [32]:
### Dividends

In [33]:
try:
    DIVIDENDS_PERPETUAL = valuation.perpetual_dividend_growth_valuation(history = Dividends_History, 
                                                        wacc=WACC)
except ZeroDivisionError:
    DIVIDENDS_PERPETUAL = 0

In [34]:
print(f"Value based on perpetual dividends growth projection: {DIVIDENDS_PERPETUAL}")

Value based on perpetual dividends growth projection: 0


# troubleshoot

In [35]:
print(f"debt weightage: {metrics.debt_weight(df, current_share_price=CURRENT_PRICE)}")
print(f"debt cost: {metrics.debt_cost(df)}")
print(f"debt cost: {metrics.debt_cost(df)}")
print("\n")
print(f"equity weightage: {metrics.equity_weight(df, current_share_price=CURRENT_PRICE)}")
print(f"equity cost: {metrics.equity_cost(market_return_rate=MARKET_RETURN,risk_free_rate=RISK_FREE_RETURN,beta=BETA)}")


debt weightage: 0.015364644205062847
debt cost: 0.09963812060594585
debt cost: 0.09963812060594585


equity weightage: 0.9846353557949371
equity cost: 0.08310000000000001


In [36]:
OCF_History

[4516683000.0, 4321323000.0, 2128900000.0, 1129798000.0]